In [1]:
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
import matplotlib.pyplot as plt
%matplotlib inline
from math import sqrt

In [11]:
data = pd.read_csv("order.txt", sep = ",", parse_dates= True, header = None , names = ['Transaction', 'Date','volumn', 'ticker'])

In [12]:
data.volumn = data.volumn.astype(float)

In [13]:
data.head()

,Transaction,Date,volumn,ticker
0,Buy,2017-01-03,100.0,AAPL
1,Buy,2017-02-15,150.0,AMZN
2,Sell,2017-04-05,100.0,AAPL
3,Sell,2017-02-07,200.0,GOOGL
4,Buy,2017-02-05,100.0,AAPL


In [14]:
pd.pivot_table(data.reset_index(), index = ['ticker', 'Transaction','Date'], values ='volumn', aggfunc= 'sum')

volumn
ticker Transaction Date              
AAPL   Buy         2016-02-05   100.0
                   2016-02-18   100.0
                   2016-05-19   100.0
                   2017-01-03   100.0
                   2017-02-05   100.0
                   2017-12-05   100.0
       Sell        2017-03-18   100.0
                   2017-04-05   100.0
                   2017-06-05   100.0
AMZN   Buy         2017-02-15   150.0
       Sell        2017-09-05   550.0
GOOGL  Buy         2017-07-10   400.0
       Sell        2017-02-07   200.0

## Find Price of all stocks each date

In [15]:
start = datetime.datetime(2016,1,1)
end = datetime.date.today()

aapl = pdr.get_data_yahoo('AAPL', start, end) 
amzn = pdr.get_data_yahoo('AMZN', start, end)
goog = pdr.get_data_yahoo('GOOGL', start, end)

In [16]:
prix_aapl = aapl['Adj Close']
prix_amzn = amzn['Adj Close']
prix_goog = goog['Adj Close']

In [17]:
prix = pd.concat([prix_aapl,prix_amzn, prix_goog], axis = 1)

In [18]:
prix.columns = ['AAPL','AMZN', 'GOOGL']

In [19]:
prix.head()

,AAPL,AMZN,GOOGL
Date,,,
2016-01-04,101.790649,636.989990,759.440002
2016-01-05,99.239845,633.789978,761.530029
2016-01-06,97.297760,632.650024,759.330017
2016-01-07,93.191338,607.940002,741.000000
2016-01-08,93.684120,607.049988,730.909973


## Add Column Price per stock to data

In [25]:
def get_close_price(ticker, date):
    date = datetime.datetime.strptime(date, '%Y-%m-%d')
    
    if date in prix.index:
        return prix[ticker][date]
    else: 
        return 0

In [26]:
data['Price'] = data.apply(lambda row: get_close_price(row['ticker'], row['Date']), axis = 1)

In [27]:
data

,Transaction,Date,volumn,ticker,Price
0,Buy,2017-01-03,100.0,AAPL,114.722694
1,Buy,2017-02-15,150.0,AMZN,842.700012
2,Sell,2017-04-05,100.0,AAPL,142.866959
3,Sell,2017-02-07,200.0,GOOGL,829.229980
4,Buy,2017-02-05,100.0,AAPL,0.000000
5,Sell,2017-06-05,100.0,AAPL,153.327881
6,Buy,2016-02-05,100.0,AAPL,91.336388
7,Buy,2017-07-10,400.0,GOOGL,951.000000
8,Sell,2017-09-05,550.0,AMZN,965.270020
9,Buy,2017-12-05,100.0,AAPL,0.000000


In [44]:
data['Montant'] = data.volumn * data.Price

In [45]:
data

,Transaction,Date,volumn,ticker,Price,Montant,sign
0,Buy,2017-01-03,100.0,AAPL,114.722694,11472.2694,-1
1,Buy,2017-02-15,150.0,AMZN,842.700012,126405.0018,-1
2,Sell,2017-04-05,100.0,AAPL,142.866959,14286.6959,1
3,Sell,2017-02-07,200.0,GOOGL,829.229980,165845.9960,1
4,Buy,2017-02-05,100.0,AAPL,0.000000,0.0000,-1
5,Sell,2017-06-05,100.0,AAPL,153.327881,15332.7881,1
6,Buy,2016-02-05,100.0,AAPL,91.336388,9133.6388,-1
7,Buy,2017-07-10,400.0,GOOGL,951.000000,380400.0000,-1
8,Sell,2017-09-05,550.0,AMZN,965.270020,530898.5110,1
9,Buy,2017-12-05,100.0,AAPL,0.000000,0.0000,-1


In [46]:
data['sign'] = data['Transaction'].apply(lambda x: -1 if x == 'Buy' else 1) 

In [47]:
data.Montant= data.Montant * data.sign

In [48]:
data

,Transaction,Date,volumn,ticker,Price,Montant,sign
0,Buy,2017-01-03,100.0,AAPL,114.722694,-11472.2694,-1
1,Buy,2017-02-15,150.0,AMZN,842.700012,-126405.0018,-1
2,Sell,2017-04-05,100.0,AAPL,142.866959,14286.6959,1
3,Sell,2017-02-07,200.0,GOOGL,829.229980,165845.9960,1
4,Buy,2017-02-05,100.0,AAPL,0.000000,-0.0000,-1
5,Sell,2017-06-05,100.0,AAPL,153.327881,15332.7881,1
6,Buy,2016-02-05,100.0,AAPL,91.336388,-9133.6388,-1
7,Buy,2017-07-10,400.0,GOOGL,951.000000,-380400.0000,-1
8,Sell,2017-09-05,550.0,AMZN,965.270020,530898.5110,1
9,Buy,2017-12-05,100.0,AAPL,0.000000,-0.0000,-1


In [49]:
data.groupby('ticker').sum()['Montant']

ticker
AAPL      -9544.5106
AMZN     404493.5092
GOOGL   -214554.0040
Name: Montant, dtype: float64